In [1]:
import os
import numpy as np 
import shutil 
import pandas as pd 

from PIL import Image
import matplotlib.pyplot as plt
import cv2 

In [2]:
root_path = '/home/wtan/Work/Datasets/PPE/chvg-pictor.v3i.yolov8' 
out_root_path = root_path+'/PPE-Images' 

splits = ['train','valid','test']

os.makedirs(root_path+'/PPE-Images/train/images', exist_ok = True) 
os.makedirs(root_path+'/PPE-Images/train/labels', exist_ok = True) 
os.makedirs(root_path+'/PPE-Images/valid/images', exist_ok = True) 
os.makedirs(root_path+'/PPE-Images/valid/labels', exist_ok = True) 
os.makedirs(root_path+'/PPE-Images/test/images', exist_ok = True) 
os.makedirs(root_path+'/PPE-Images/test/labels', exist_ok = True) 

In [3]:
nc = 5
names =  ['glass', 'head', 'helmet', 'person', 'vest']
person_id = 3 
helmet_id = 2 
vest_id = 4 

helmet_id_new = 0
vest_id_new = 1 

In [87]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

def bb_A_inside_B(boxA, boxB, margin=10):
    return boxA[0]>=boxB[0]-margin and boxA[1]>=boxB[1]-margin and boxA[2]<=boxB[2]+margin and boxA[3]<=boxB[3]+margin 


In [103]:
debug = False 

for split in ['train','valid','test']:
    
    imgfiles = os.listdir(os.path.join(root_path, split, 'images')) 
    imgfiles = [x for x in imgfiles if x.split('.')[-1].lower() in ['jpg','jpeg','png']] 
    imgfiles.sort()     
    #print(len(imgfiles),imgfiles[:5]) 

    for f in imgfiles:

        #if not f == '000006_jpg.rf.71d3a78d37b2eb53e50eb1641a374e4d.jpg':
        #    continue 

        fname = os.path.splitext(f)[0]  
        img_file = os.path.join(os.path.join(root_path, split, 'images',f))          
        label_file = os.path.join(os.path.join(root_path, split, 'labels',fname+'.txt'))
        print(img_file) 

        fp = open(label_file,'r')
        labels = fp.readlines() 
        fp.close() 
        labels = [x.strip() for x in labels] 
        if debug: 
            print("labels = ", labels) 

        im = Image.open(img_file)
        width, height = im.size

        if debug: 
            plt.imshow(im) 
            plt.show() 

        person_bboxs = [] 
        helmet_bboxs = [] 
        vest_bboxs = [] 

        for l in labels: 
            l = l.split(' ') 
            l = [float(x) for x in l] 

            if l[0]==0 or l[0]==1: #class 
                continue 

            w = int(l[3]*width) 
            h = int(l[4]*height) 
            x = int((l[1]-l[3]/2)*width)             
            y = int((l[2]-l[4]/2)*height) 

            if debug: 
                print(l,w,h,x,y)                 
                
            #im_person = im[y:y+h,x:x+w] #opencv or numpy 
            im_crop = im.crop((x,y,x+w,y+h))

            if l[0] == person_id:                                
                person_bboxs.append((x,y,x+w,y+h))
            elif l[0]== helmet_id:
                helmet_bboxs.append((x,y,x+w,y+h))
            elif l[0]== vest_id:
                vest_bboxs.append((x,y,x+w,y+h))

        #print(bboxs_by_person) 
        for i in range(len(person_bboxs)): 
            pbox = person_bboxs[i] 
            im_person = im.crop(pbox)
            if debug: 
                plt.imshow(im_person) 
                plt.title('person')
                plt.show() 
                print("person bbox = ", pbox) 

            #generate bbox of helmet and vest in the person image  
            width, height = im_person.size
            #print('person image size = ', width, height) 

            labels = [] 
            for hbox in helmet_bboxs:                
                if not bb_A_inside_B(hbox,pbox):
                    #print('hbox inside pbox = False') 
                    continue 
                elif debug:
                    print("pbox = ", pbox) 
                    print("Find match hbox = ", hbox) 
                    
                left  = hbox[0]-pbox[0]
                upper = hbox[1]-pbox[1]
                right = hbox[2]-pbox[0] 
                lower = hbox[3]-pbox[1] 
                x = (left+right)/2/width 
                y = (upper+lower)/2/height 
                w = (right-left)/width 
                h = (lower-upper)/height 

                if debug: 
                    print("hbox = ", hbox) 
                    print((x,y,w,h))
                    im_helmet = im.crop(hbox)
                    plt.imshow(im_helmet) 
                    plt.title('helmet') 
                    plt.show() 

                #there is still possible mismatched helmet and person 
                if x<0 or x>0.99 or y<0 or y>0.99 or w<0 or w>0.99 or h<0 or h>0.99:
                    continue 
                
                labels.append((helmet_id_new,x,y,w,h)) 
                

            #print("debug vbox, pbox = ", pbox)
            for vbox in vest_bboxs:
                if not bb_A_inside_B(vbox,pbox):
                    #print(vbox) 
                    #print('vbox inside pbox = False') 
                    continue 
                elif debug:
                    print("pbox = ", pbox) 
                    print("Find match vbox = ", vbox) 
                    
                left  = vbox[0]-pbox[0]
                upper = vbox[1]-pbox[1]
                right = vbox[2]-pbox[0] 
                lower = vbox[3]-pbox[1] 
                
                x = (left+right)/2/width 
                y = (upper+lower)/2/height 
                w = (right-left)/width 
                h = (lower-upper)/height 
                if debug: 
                    #print("vbox = ", vbox) 
                    #print((x,y,w,h))
                    im_vest = im.crop(vbox)
                    plt.imshow(im_vest) 
                    plt.title('vest') 
                    plt.show() 

                #there is still possible mismatched helmet and person 
                if x<0 or x>0.99 or y<0 or y>0.99 or w<0 or w>0.99 or h<0 or h>0.99:
                    continue 
                    
                labels.append((vest_id_new,x,y,w,h)) 

            #write images and label files 
            out_imgfile = fname + '_{:03d}.jpg'.format(i) 
            out_labelfile = fname + '_{:03d}.txt'.format(i)

            out_imgfile = os.path.join(out_root_path,split,'images',out_imgfile) 
            out_labelfile = os.path.join(out_root_path,split,'labels',out_labelfile)

            #print(out_imgfile) 
            #print(out_labelfile) 
            im_person.save(out_imgfile)
            if debug:
                print("labels = ", labels) 
            fp = open(out_labelfile,'w') 
            for l in labels:
                fp.write('{} {} {} {} {}\n'.format(l[0],l[1],l[2],l[3],l[4])) 
            fp.close() 

/home/wtan/Work/Datasets/PPE/chvg-pictor.v3i.yolov8/train/images/000004_jpg.rf.1485a9a59604e5aebbf67aa3eb27074e.jpg /home/wtan/Work/Datasets/PPE/chvg-pictor.v3i.yolov8/train/labels/000004_jpg.rf.1485a9a59604e5aebbf67aa3eb27074e.txt
/home/wtan/Work/Datasets/PPE/chvg-pictor.v3i.yolov8/train/images/000004_jpg.rf.e42c9e18e431030d91681830032a10c1.jpg /home/wtan/Work/Datasets/PPE/chvg-pictor.v3i.yolov8/train/labels/000004_jpg.rf.e42c9e18e431030d91681830032a10c1.txt
/home/wtan/Work/Datasets/PPE/chvg-pictor.v3i.yolov8/train/images/000004_jpg.rf.e994f37da95d4fecd3f98d8299578abb.jpg /home/wtan/Work/Datasets/PPE/chvg-pictor.v3i.yolov8/train/labels/000004_jpg.rf.e994f37da95d4fecd3f98d8299578abb.txt
/home/wtan/Work/Datasets/PPE/chvg-pictor.v3i.yolov8/train/images/000006_jpg.rf.670d0c20daa9123f986ed8ea1aff34d2.jpg /home/wtan/Work/Datasets/PPE/chvg-pictor.v3i.yolov8/train/labels/000006_jpg.rf.670d0c20daa9123f986ed8ea1aff34d2.txt
/home/wtan/Work/Datasets/PPE/chvg-pictor.v3i.yolov8/train/images/000006_

In [4]:
def draw_bbox_on_image(jpg_fn,label_fn):
    
    #print(jpg_fn)
    #print(label_fn) 

    im = cv2.imread(jpg_fn) 
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    height,width = im.shape[:2] 
    longer_edge = max(height,width) 
    scale = 640/longer_edge 
    im = cv2.resize(im, (0, 0), fx = scale, fy = scale)
    height,width = im.shape[:2] 
    
    #print(im.shape) 

    # Blue color in BGR
    color = (0, 0, 255)
    # Line thickness of 2 px
    thickness = 1
    
    lines = open(label_fn,"r").readlines()     
    lines = [x.strip() for x in lines] 

    #print(lines) 

    classes = [] 
    
    for l in lines: 
        annot = l.split(' ') 
        cls = int(annot[0]) 

        w = float(annot[3])
        h = float(annot[4])

        #print(cls,w,h) 

        classes.append(cls) 
        
        if w<0.005 or h<0.005: 
            print(cls,w,h) 

        w = int(w*width)
        h = int(h*height)

        x1 = int(float(annot[1])*width)-w//2
        y1 = int(float(annot[2])*height)-h//2 
        x2 = x1 + w 
        y2 = y1 + h

        im = cv2.rectangle(im, (x1,y1),(x2,y2), color, thickness) 


    cv2.imshow('object',im) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return classes 


In [6]:
sp = 'valid'
img_path = os.path.join(out_root_path,sp,'images') 
label_path = os.path.join(out_root_path,sp,'labels') 
img_fns = os.listdir(img_path) 
print(len(img_fns))

1212


In [7]:
for i,jpg in enumerate(img_fns): 

    print(jpg) 
    
    jpg_fn = os.path.join(img_path,jpg)
    
    label_fn = os.path.join(label_path,jpg.replace('.jpg','.txt')) 
    if i<100:
        draw_bbox_on_image(jpg_fn,label_fn)

ppe_1296_jpg.rf.1efb0f4947592af177a6c64872bb8f7b_001.jpg
ppe_0604_jpg.rf.12823b2f5dac33d341705b4143ece932_012.jpg
gettyimages-1177326209-612x612_jpg.rf.babc405679430b1759a8fd4a37daab34_004.jpg
part2_000246_jpg.rf.6113ec0595fa65afb94e0f1ac989650c_000.jpg
ppe_0068_jpg.rf.b6b5b3e468aee5213db822e1a3a85942_002.jpg
ppe_1035_jpg.rf.c6d154657c0aab2c3925a4e0d54245eb_001.jpg
ppe_0102_jpg.rf.b5603553b0cdb00c653e98468bc2e24c_000.jpg
ppe_0493_jpg.rf.453faa1fab3cfb54366a59a97dbb7cc9_000.jpg
image_from_china-1352-_jpg.rf.0157967a95ec4034d1b4b2db14570d4e_001.jpg
image_from_china-3285-_jpg.rf.29c78fc6fa2d21daf45e011977e20306_001.jpg
ppe_0282_jpg.rf.e1aca6b7d3dba0e8151ce77a6e41c02a_003.jpg
image_from_china-4328-_jpg.rf.108ca7079d480079f417c562b65e3528_001.jpg
ppe_0754_jpg.rf.0907d81725c9851d8d132f3de27e159e_004.jpg
ppe_0148_jpg.rf.201f25727737142d362930e17251ed1f_008.jpg
ppe_0258_jpg.rf.249ec8aa0c3943a64a405bcae1010222_000.jpg
ppe_1035_jpg.rf.c6d154657c0aab2c3925a4e0d54245eb_006.jpg
ppe_1163_jpg.rf.52b8